Import the modules we will use for this notebook.

In [1]:
import numpy as np
from numpy.linalg import inv
from numpy.linalg import det
from numpy.linalg import matrix_rank
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# The below suppresses all warnings in the notebook
# Only leave this uncommented for display purposes
import warnings
warnings.filterwarnings("ignore")

Now load the data into memory.

In [2]:
df = pd.read_csv('OLS_Data.csv')

df.head()

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,y
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,0.00632,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,0.02731,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,0.02729,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,0.03237,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,0.06905,36.2


Check some basic information about the columns in the dataset

In [3]:
df.info()

print('\n')
print("Dimension of the data: ", df.shape)

no_of_rows = df.shape[0]
no_of_columns = df.shape[1]

print("No. of Rows: %d" % no_of_rows)
print("No. of Columns: %d" % no_of_columns)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 15 columns):
X1     506 non-null float64
X2     506 non-null float64
X3     506 non-null float64
X4     506 non-null int64
X5     506 non-null float64
X6     506 non-null float64
X7     506 non-null float64
X8     506 non-null float64
X9     506 non-null int64
X10    506 non-null int64
X11    506 non-null float64
X12    506 non-null float64
X13    506 non-null float64
X14    506 non-null float64
y      506 non-null float64
dtypes: float64(12), int64(3)
memory usage: 59.4 KB


Dimension of the data:  (506, 15)
No. of Rows: 506
No. of Columns: 15


Check the colinearity of each feature will all other features.<br>
Note that if the colinearity is too close to 1 that feature is likely redundant and can be removed from the model.

In [4]:
df.corr()

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,y
X1,1.000000,-0.200469,0.406583,-0.055892,0.420972,-0.219247,0.352734,-0.379670,0.625505,0.582764,0.289946,-0.385064,0.455621,1.000000,-0.388305
X2,-0.200469,1.000000,-0.533828,-0.042697,-0.516604,0.311991,-0.569537,0.664408,-0.311948,-0.314563,-0.391679,0.175520,-0.412995,-0.200469,0.360445
X3,0.406583,-0.533828,1.000000,0.062938,0.763651,-0.391676,0.644779,-0.708027,0.595129,0.720760,0.383248,-0.356977,0.603800,0.406583,-0.483725
X4,-0.055892,-0.042697,0.062938,1.000000,0.091203,0.091251,0.086518,-0.099176,-0.007368,-0.035587,-0.121515,0.048788,-0.053929,-0.055892,0.175260
X5,0.420972,-0.516604,0.763651,0.091203,1.000000,-0.302188,0.731470,-0.769230,0.611441,0.668023,0.188933,-0.380051,0.590879,0.420972,-0.427321
X6,-0.219247,0.311991,-0.391676,0.091251,-0.302188,1.000000,-0.240265,0.205246,-0.209847,-0.292048,-0.355501,0.128069,-0.613808,-0.219247,0.695360
X7,0.352734,-0.569537,0.644779,0.086518,0.731470,-0.240265,1.000000,-0.747881,0.456022,0.506456,0.261515,-0.273534,0.602339,0.352734,-0.376955
X8,-0.379670,0.664408,-0.708027,-0.099176,-0.769230,0.205246,-0.747881,1.000000,-0.494588,-0.534432,-0.232471,0.291512,-0.496996,-0.379670,0.249929
X9,0.625505,-0.311948,0.595129,-0.007368,0.611441,-0.209847,0.456022,-0.494588,1.000000,0.910228,0.464741,-0.444413,0.488676,0.625505,-0.381626
X10,0.582764,-0.314563,0.720760,-0.035587,0.668023,-0.292048,0.506456,-0.534432,0.910228,1.000000,0.460853,-0.441808,0.543993,0.582764,-0.468536


X1 and X14 seem to be duplicates, and can be removed.

We now remove X1 and X14 from our dataset.

In [5]:
df = df.drop(columns=['X1','X14'])

Now separate data into the feature matrix and the target vector.

In [6]:
df_X = df.drop(columns='y',axis=1)
X=df_X.values # Turn into numpy array

df_Y = df['y']
y = df_Y.values # Turn into numpy array

print(X.shape)
print(y.shape)

(506, 12)
(506,)


We now scale our feature matrix.

In [7]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

Now create the testing and training datasets.

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)

We now use the built-in module for performing linear regression for the dataset.<br>
The below output shows the results for the training data.

In [9]:
# Create the sklearn OLS linear regression object
lin_reg = LinearRegression()

# Train the model
lin_reg.fit(X_train, y_train)

# Print the calculated optimal intercept
print("Intercept: \n", lin_reg.intercept_)

# Print the calculated optimal coefficients
print("Coefficients: \n", lin_reg.coef_)

print("\n----------------------------- Model Evaluation for training data -----------------------------")

# Make prediction on the training data
y_train_predicted = lin_reg.predict(X_train)

print("\nMean squared error: %.2f" % mean_squared_error(y_train, y_train_predicted))

# Explained variance score: 1 is perfect prediction
print("Coefficient of determination r^2 variance score [1 is perfect prediction]: %.2f" % r2_score(y_train, y_train_predicted))

# Explained variance score: 1 is perfect prediction
print("Coefficient of determination r^2 variance score [1 is perfect prediction]: %.2f" % lin_reg.score(X_train, y_train))

Intercept: 
 22.42334612898126
Coefficients: 
 [ 0.66844983  0.05527183  0.92138931 -1.07916096  3.8233023  -0.87614882
 -2.61829801  1.3703187  -1.929301   -1.55273219  1.1051149  -3.05626349]

----------------------------- Model Evaluation for training data -----------------------------

Mean squared error: 17.03
Coefficient of determination r^2 variance score [1 is perfect prediction]: 0.79
Coefficient of determination r^2 variance score [1 is perfect prediction]: 0.79


We now see how well the model fits with the testing data.

In [10]:
# Make prediction 
y_test_predicted = lin_reg.predict(X_test)

print('Below is for the test data...')

print("Mean squared error: %.2f" % mean_squared_error(y_test, y_test_predicted))

# Explained variance score: 1 is perfect prediction
print("Coefficient of determination r^2 variance score [1 is perfect prediction]: %.2f" % r2_score(y_test, y_test_predicted))

Below is for the test data...
Mean squared error: 48.83
Coefficient of determination r^2 variance score [1 is perfect prediction]: 0.53


This is not a very good fit, but as we forced our data to fit a linear model, while it may not, this is not to be entirely unexpected.

In the below box we now perform the same calculation as was done above for fitting to a linear model, but we now show every step of the calculation.<br>
The below output shows how well the model works for the training data.

In [11]:
# Add a bias term with the feature vectors to create a new data matrix "X_train_bias"
X_train_bias = X_bias = np.c_[np.ones((X_train.shape[0],1)),X_train]

# Print the determinant of the dot product of the transpose of X_train_bias and X_train_bias
print("\nDeterminant of (X_train_bias^T.X_train_bias): ", det(X_train_bias.T.dot(X_train_bias)))

# Computes the dot product of the transpose of X_train_bias with itself
#  Denote the product as "z"
z = X_train_bias.T.dot(X_train_bias)

# Closed form (OLS) solution for weight vector w 
w = np.linalg.inv(z).dot(X_train_bias.T).dot(y_train)

print("\nThe weight vector:\n",w)

print("\n----------------------------- Model Evaluation for training data -----------------------------")

# Make prediction using the X_train_bias data matrix
# The predicted target vector should be named as "y_train_predicted"
y_train_predicted = X_train_bias.dot(w)

# Compute the MSE
print("Mean squared error: %.2f" % mean_squared_error(y_train, y_train_predicted))

# Compute the r^2 score
print("Coefficient of determination r^2 variance score [1 is perfect prediction]: %.2f" % r2_score(y_train, y_train_predicted))


Determinant of (X_train_bias^T.X_train_bias):  1.0903515290696932e+30

The weight vector:
 [22.42334613  0.66844983  0.05527183  0.92138931 -1.07916096  3.8233023
 -0.87614882 -2.61829801  1.3703187  -1.929301   -1.55273219  1.1051149
 -3.05626349]

----------------------------- Model Evaluation for training data -----------------------------
Mean squared error: 17.03
Coefficient of determination r^2 variance score [1 is perfect prediction]: 0.79


As expected, this returns the exact same output as we obtained from the built-in approach.

For the above calculation there were no problems, and we were able to calculate the final model coefficients.<br>
However, there can become a problem if we have a matrix such that we cannot calculate its inverse exactly.<br>
For that situation we can implement an approach such as that discussed below.<br>
Below we implement a simplified for of Ridge Regression, where we manually add a small number to the diagonal of our feature matrix to ensure that it will now be possible to invert the matrix.

In [12]:
# Add a bias term with the feature vectors to create a new data matrix "X_train_bias"
X_train_bias = np.c_[np.ones((X_train.shape[0],1)),X_train]

# Print the determinant of the dot product of the transpose of X_train_bias and X_train_bias
print("\nDeterminant of (X_train_bias^T.X_train_bias): ", det(X_train_bias.T.dot(X_train_bias)))

# Computes the dot product of the transpose of X_train_bias with itself
z = X_train_bias.T.dot(X_train_bias)

print("\n-------- Fixing the Singularity of (X_bias^T).X_bias ------------")

# Create a diagonal matrix that has the dimension of z"
diagonal = np.zeros(z.shape, float)

# Add small positive non-zero numbers on the diagonal
np.fill_diagonal(diagonal, 0.001)

print("The diagonal matrix:\n", diagonal)

# Closed form (OLS) solution for weight vector w
w = np.linalg.inv(z + diagonal).dot(X_train_bias.T).dot(y_train)

print("\nThe weight vector:\n",w)

print("\n----------------------------- Model Evaluation for the training data -----------------------------")

# Make prediction using the X_train_bias data matrix
y_train_predicted = X_train_bias.dot(w)

# Compute the MSE
print("Mean squared error: %.2f" % mean_squared_error(y_train, y_train_predicted))

# Compute the r^2 score
print("Coefficient of determination r^2 variance score [1 is perfect prediction]: %.2f" % r2_score(y_train, y_train_predicted))


Determinant of (X_train_bias^T.X_train_bias):  1.0903515290696932e+30

-------- Fixing the Singularity of (X_bias^T).X_bias ------------
The diagonal matrix:
 [[0.001 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
  0.   ]
 [0.    0.001 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
  0.   ]
 [0.    0.    0.001 0.    0.    0.    0.    0.    0.    0.    0.    0.
  0.   ]
 [0.    0.    0.    0.001 0.    0.    0.    0.    0.    0.    0.    0.
  0.   ]
 [0.    0.    0.    0.    0.001 0.    0.    0.    0.    0.    0.    0.
  0.   ]
 [0.    0.    0.    0.    0.    0.001 0.    0.    0.    0.    0.    0.
  0.   ]
 [0.    0.    0.    0.    0.    0.    0.001 0.    0.    0.    0.    0.
  0.   ]
 [0.    0.    0.    0.    0.    0.    0.    0.001 0.    0.    0.    0.
  0.   ]
 [0.    0.    0.    0.    0.    0.    0.    0.    0.001 0.    0.    0.
  0.   ]
 [0.    0.    0.    0.    0.    0.    0.    0.    0.    0.001 0.    0.
  0.   ]
 [0.    0.    0.    0.    0.    0.    0.

As expected, this gives nearly the same result, with only minor variations in the discovered coefficients.

We now apply this Ridge Regression technique to our testing dataset.

In [13]:
# Use weights calculated above using the Ridge Regression method

X_test_bias = np.c_[np.ones((X_test.shape[0],1)),X_test]

y_test_predicted = X_test_bias.dot(w)
print('Below is for the test data...')

# Compute the MSE
print("Mean squared error: %.2f" % mean_squared_error(y_test, y_test_predicted))

# Compute the r^2 score
print("Coefficient of determination r^2 variance score [1 is perfect prediction]: %.2f" % r2_score(y_test, y_test_predicted))

Below is for the test data...
Mean squared error: 48.83
Coefficient of determination r^2 variance score [1 is perfect prediction]: 0.53


This shows a fit just as good as that calculated previously.